# Sistema de recomendación de películas

# Marina Fernández

El gran problema para realizar este trabajo fue, sin duda, los pocos datos que las diferentes APIs proporcionaban de forma libre y gratuita: las que proporcionaban ids de usuarios no proporcionaban puntuación y viceversa.
Así, la realización de un motor de recomendación con collaborative filtering, el que fuera mi primer objetivo, dejó paso a la elaboración de un algoritmo de creación propia para recomendar películas al usuario a partir de una que este mismo proporciona.

Para el dataset, realizamos una llamada a la API de "The Movie DB" y nos traemos la información de 500 páginas con la ayuda de un bucle "for".
En particular, necesitaremos los títulos de las películas, sus puntuaciones y una breve descripción en el caso de que queramos acceso a más detalle.
Guardaremos estos datos en las listas "titles","rates","genres" y "desc" respectivamente siempre y cuando el recuento total de votos sea superior a 1000 para quitarnos películas con un bajo número de votos que suelen tender a tener resultados menos "realistas".


In [82]:
import urllib.parse
import urllib.request
from urllib.request import urlopen
import pandas as pd
from collections import Counter
import json

api_key = "cf0e8e8b30fe4f327d670417d6e8eee0"
titles=[]
rates=[]
genres=[]
desc=[]

for k in range(1,500):
    page_number = k
    
    url = "https://api.themoviedb.org/3/discover/movie?api_key="+api_key+"&language=en-US&sort_by=popularity.desc&include_adult=false&include_video=false&page="+str(page_number)
    data = urllib.request.urlopen(url).read().decode()
    js = json.loads(data)

    for result in js["results"]:
        if result["vote_count"]>1000:
            print(result["title"])
            titles.append(result["title"])
            rates.append(float(result["vote_average"]))
            genres.append(result["genre_ids"])
            desc.append(result["overview"])
            

Wonder Woman 1984
Soul
Tenet
Ava
Birds of Prey (and the Fantabulous Emancipation of One Harley Quinn)
The Croods: A New Age
Greenland
Bad Boys for Life
Godzilla: King of the Monsters
Extraction
Bloodshot
Joker
The SpongeBob Movie: Sponge on the Run
Fast & Furious Presents: Hobbs & Shaw
Jumanji: The Next Level
After We Collided
Avengers: Infinity War
Mulan
Code 8
Roald Dahl's The Witches
Avengers: Endgame
The New Mutants
Spider-Man: Far from Home
Terminator: Dark Fate
Underwater
Peninsula
6 Underground
To All the Boys: P.S. I Still Love You
After
Venom
Captain Marvel
John Wick: Chapter 3 - Parabellum
Star Wars: The Rise of Skywalker
Gemini Man
Spenser Confidential
To All the Boys I've Loved Before
Dark Phoenix
Charlie's Angels
The Meg
Zombieland: Double Tap
Kong: Skull Island
Angel Has Fallen
Harry Potter and the Philosopher's Stone
Rambo: Last Blood
Justice League
Ready Player One
Shazam!
Parasite
Ford v Ferrari
Thor: Ragnarok
Rampage
Twilight
Crawl
The Gentlemen
Anna
Harry Potter and 

Una vez que tenemos nuestro datos, creamos el Dataframe haciendo un "zip" de las listas dónde introdujimos la 
información y lo "limpiamos", es decir, eliminamos las filas en las que los títulos estén duplicados.
Además, pasamos estos títulos a minúsculas para facilitar la futura búsqueda.

In [86]:
df = pd.DataFrame(list(zip(titles,rates,genres,desc)), 
               columns =['Title','Rate','Genres','Description']) 

print("This is our dataframe:")
pd.set_option("display.max_colwidth", None)
display(df.head())
df["Title"] = df["Title"].str.lower()
df = df.drop_duplicates(['Title'], keep='first')


This is our dataframe:


,Title,Rate,Genres,Description
0,Wonder Woman 1984,6.9,"[14, 28, 12]",Wonder Woman comes into conflict with the Soviet Union during the Cold War in the 1980s and finds a formidable foe by the name of the Cheetah.
1,Soul,8.3,"[10751, 16, 35, 18, 10402, 14]","Joe Gardner is a middle school teacher with a love for jazz music. After a successful gig at the Half Note Club, he suddenly gets into an accident that separates his soul from his body and is transported to the You Seminar, a center in which souls develop and gain passions before being transported to a newborn child. Joe must enlist help from the other souls-in-training, like 22, a soul who has spent eons in the You Seminar, in order to get back to Earth."
2,Tenet,7.3,"[28, 53, 878]","Armed with only one word - Tenet - and fighting for the survival of the entire world, the Protagonist journeys through a twilight world of international espionage on a mission that will unfold in something beyond real time."
3,Ava,5.6,"[28, 80, 18, 53]",A black ops assassin is forced to fight for her own survival after a job goes dangerously wrong.
4,Birds of Prey (and the Fantabulous Emancipation of One Harley Quinn),7.1,"[28, 80]","Harley Quinn joins forces with a singer, an assassin and a police detective to help a young girl who had a hit placed on her after she stole a rare diamond from a crime lord."


In [87]:
print(len(df.index))

2594


El resultado es un dataframe con más de 2500 filas, es decir, nuestro estudios se basará en una muestra de más de 2500 películas, entre las cuales intentaremos buscar aquella que mejor se adapte a los gustos del usuario.

En un primer momento, y tras una breve presentación de intenciones, le preguntaremos a nuestro "interlocutor" por
una película que le guste especialmente.
Si el programa no encuentra este título entre aquellos en mi base de datos, repetiremos la operación hasta 
conseguirlo.
Conseguimos esto gracias a un "while" y un "break" que rompe el bucle cuando, una vez aplicada la máscara del título
a mi dataframe, este tiene un tamaño superior a 0, es decir, no está vacío: nos aseguramos así de disponer de la
información suficiente para proseguir.

Una vez que ya tenemos la película, rompemos nuestro dataframe en uno nuevo filtrado por esta: esto nos 
facilitara el trabajo a la hora de acceder a la información.
Así, asignamos a la variable filmRate la puntuación media de la película elegida y a la variable filmGenres
la lista de géneros de esta e imprimimos por pantalla.

A continuación, filtramos nuestro dataframe original para encontrar aquellas películas que, en valor absoluto,
tienen una puntuación que diste de la película elegida de menos de 0.3 y le asignamos el nombre similarRated.
Elegimos este valor de manera aleatoria.
Reseteamos el index de similarRated ya que anteriormente los índices de este coincidian con los índices de nuestro
dataframe original (si la primera película filtrada tenía índice 10 seguirá teniendolo)y borramos los antiguos.
Estos índices serán indispensables a la hora de buscar los diferentes títulos en mi nueva tabla.
Para terminar esta parte, imprimimos por pantalla la longitud del nuevo índice, que coincide con la cantidad de películas que encontramos en nuestra database y que cumplen la condición que hemos establecido previamente.

Llegamos a la parte principal: empezamos definiendo nuestra variable lista recommendations, dónde iremos añadiendo
una a una las diferentes películas recomendadas y lanzamos la primera búsqueda.
Con la ayuda de un bucle for, comprobamos si la lista de géneros de la película que nos proporciono el interlocutor
coincide exactamente con la lista de géneros de las películas en nuestro dataframe similarRated.
En el caso de que coincidan exactamente y de que la película no es aquella que se nos proporciona, añadimos
el o los diferentes títulos a nuestra lista de recomendaciones.

En el caso de que encontremos más de un título, imprimiremos estos por pantalla, si sólo encontramos una película, imprimiremos este por pantalla y en el caso de que no encontremos ninguna que coincida exactamente probaremos suerte siguiendo un nuevo criterio.

Para este último caso, es decir, para el caso en el que la lista recommendations esté vacía,recomendaremos otros títulos que, posiblemente, no se adapten a los gustos del consumidor de una manera tan exacta pero que igualmente podrían adaptarse a estos de una manera bastante fiable. 
Lanzamos para estos, una nueva búsqueda dónde, sin buscar que los géneros coincidan exactamente, añadimos a una nueva variable lista pruebas los diferentes títulos de las películas cuyos géneros aparezcan entre los géneros de la peli dada siempre y cuando este no vuelva a coincidir con la que nos han dado.
Para explicarnos, si encuentro una película en similarRated con 3 géneros de los 4 de la película elegida, este título será añadido a la lista pruebas 3 veces.
De entre estos, añadiremos el que más se repite a nuestra variable recommendations e imprimiremos esta nueva recomendación por pantalla. 
Al añadir aquel que más se repite, nos aseguramos de que sea la película que más géneros comparte con la película original y por lo tanto, la que más posibilidades tiene.

Ya hemos cumplido nuestro objetivo principal: recomendar la o las películas que pensamos que más podrían gustarle
al interlocutor, sin embargo ¿por qué recomendar sólo a partir de una película y no darle la oportunidad al consumidor
de repetir este proceso las veces que desee?
Efectuamos así la pregunta cuya respuesta answer determinara si lanzamos de nuevo todo el algoritmo o si, en el caso 
de ser una respuesta negativa, damos por terminado el ejercicio y por consecuente, roto el ciclo.





In [88]:
print("\033[1mHello! My name is Marina and I will try to find the perfect film for you!:)\nTell me about yourself.\033[0m")

film=input("Can you write down a film that you love? ").lower()
  
while df[df['Title']==film].size==0 :  
    print("Please try another one, I can not find this film in my database")
    film=input("Can you tell me a film that you love? ").lower()  
      
    if(df[df['Title']==film].size>0):  
        break
        
print("The film you chose is \033[1m"+film+"\033[0m")

df_film=df[df['Title']==film]
# display(df_film)
filmRate = float(df_film['Rate'])
filmGenres = sorted(list(df_film['Genres'])[0])
print("The average rate of this movie is\033[1m",filmRate)

similarRated = df[abs(df['Rate']-filmRate)<0.3]
similarRated = similarRated.reset_index()
similarRated = similarRated.drop(['index'],axis=1)
print("\033[0mI could find a total of\033[1m",len(similarRated.index),"\033[0mfilms with a similar average rate than your film.")

recommendations=[]
for i in range(len(similarRated.index)):
    if sorted(similarRated['Genres'][i])==filmGenres and similarRated.iloc[i]['Title']!=film :
        recommendations.append(similarRated.iloc[i]['Title'])
        
print("I could find\033[1m",len(recommendations),"\033[0mfilms with the exact same gender(s) as yours and with a similar rate.")
if len(recommendations)>1:

    print("\033[1mI strongly recommend you to watch any of the following films:\033[0m")  
    for recommendation in recommendations:
        print('\033[1m'+ recommendation+'\033[0m')
elif len(recommendations)==1:
    print("\033[1mI strongly recommend you to watch the following film:"+recommendations[0]+"\033[0m")
    
else:
    print("\033[1mI am sorry I couldn't find a perfect match for you. I still recommend you the following movie: \033[0m")
    
    pruebas=[]
    for i in range(len(similarRated.index)):
        for k in range(len(similarRated['Genres'][i])):
            if similarRated['Genres'][i][k] in filmGenres and similarRated.iloc[i]['Title']!=film :
                pruebas.append(similarRated.iloc[i]['Title'])

    recommendations.append(Counter(pruebas).most_common()[0][0])
    print("\033[1m"+recommendations[0]+'\033[0m')

answer=input("Do you want to try another film? yes/no: ").lower()
    
while answer=="yes":
    film=None
    film=input("Can you tell me a film that you love? ").lower()
  
    while df[df['Title']==film].size==0 :  
        print("Please try another one, I can not find this film in my database")
        film=input("Can you tell me a film that you love? ").lower()  
      
        if(df[df['Title']==film].size>0):  
            break
    
    print("The film you chose is \033[1m"+film+"\033[0m")
    df_film=df[df['Title']==film]
    # display(df_film)
    filmRate = float(df_film['Rate'])
    filmGenres = sorted(list(df_film['Genres'])[0])
    print("The average rate of this movie is\033[1m",filmRate)


    similarRated = df[abs(df['Rate']-filmRate)<0.3]
    similarRated = similarRated.reset_index()
    similarRated = similarRated.drop(['index'],axis=1)

    print("\033[0mI could find a total of\033[1m",len(similarRated.index),"\033[0mfilms with a similar average rate than your film.")


    recommendations=[]
    for i in range(len(similarRated.index)):
        if sorted(similarRated['Genres'][i])==filmGenres and similarRated.iloc[i]['Title']!=film :

            recommendations.append(similarRated.iloc[i]['Title'])
    print("I could find\033[1m",len(recommendations),"\033[0mfilms with the exact same gender(s) as yours and with a similar rate.")
    if len(recommendations)>1:

        print("\033[1mI strongly recommend you to watch any of the following films:\033[0m")  
        for recommendation in recommendations:
            print('\033[1m'+ recommendation+'\033[0m')
    elif len(recommendations)==1:
        print("\033[1mI strongly recommend you to watch the following film:"+recommendations[0]+"\033[0m")

    else:
        print("\033[1mI am sorry I couldn't find a perfect match for you. I still recommend you the following movie: \033[0m")

        pruebas=[]
        for i in range(len(similarRated.index)):
            for k in range(len(similarRated['Genres'][i])):
                if similarRated['Genres'][i][k] in filmGenres and similarRated.iloc[i]['Title']!=film :
                    pruebas.append(similarRated.iloc[i]['Title'])

        recommendations.append(Counter(pruebas).most_common()[0][0])
        print("\033[1m"+recommendations[0]+'\033[0m')
    answer=None
    answer=input("Do you want to try with another film? yes/no: ").lower()

    if answer=="no":
        break

if answer=="no":
    print("\033[1mI hope I helped you!\033[0m")
    
    

Hello! My name is Marina and I will try to find the perfect film for you!:)
Tell me about yourself.
Can you write down a film that you love? toy story
The film you chose is toy story
The average rate of this movie is 7.9
I could find a total of 285 films with a similar average rate than your film.
I could find 3 films with the exact same gender(s) as yours and with a similar rate.
I strongly recommend you to watch any of the following films:
up
zootopia
fantastic mr. fox
Do you want to try another film? yes/no: yes
Can you tell me a film that you love? soul
The film you chose is soul
The average rate of this movie is 8.3
I could find a total of 127 films with a similar average rate than your film.
I could find 0 films with the exact same gender(s) as yours and with a similar rate.
I am sorry I couldn't find a perfect match for you. I still recommend you the following movie: 
coco
Do you want to try with another film? yes/no: no
I hope I helped you!
